<a href="https://colab.research.google.com/github/Mohamed-Harish-M-123/Capstone-Project-III/blob/main/Coronavirus_Tweets_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Project Name: Coronavirus Tweets Sentiment Analysis**
####**Project Type:** Machine Learning - Classification
####**Contribution:** Individual
####**Name:** MOHAMED HARISH


##**GitHub Link:** https://github.com/Mohamed-Harish-M-123/Capstone-Project-III

## **Business context:**
####This challenge asks you to build a classification model to predict the sentiment of covid-19 tweets. The tweets have been pulled from Twitter and manual tagging has been done then. The names and usernames have been given codes to avoid any privacy concerns.

##**Problem statement:**
####In the context of analyzing COVID-19 tweets, the task is to develop a sentiment classification model that accurately predicts the sentiment expressed in these tweets. Manual tagging has been performed on the tweets to assign sentiment labels. However, due to the vast amount of data generated on social media platforms like Twitter, manual analysis becomes time-consuming and inefficient. Therefore, there is a need to automate the sentiment analysis process for COVID-19 tweets to provide timely insights and understand public sentiment during the pandemic.
####The challenge lies in building a robust and accurate classification model that can handle the nuances of the language used in these tweets, account for privacy concerns by encoding names and usernames, and effectively classify tweets into positive, negative, or neutral sentiment categories.
####The successful development of such a model will enable organizations to gain real-time insights into public sentiment regarding COVID-19, allowing them to make informed decisions and respond effectively to public concerns and needs.